# RAG - Retrieval & Search


Install Packages


In [1]:
!uv pip install -q \
    python-dotenv==1.2.1 \
    pandas==2.3.2 \
    pandas-stubs==2.3.2.250827 \
    numpy==2.3.2 \
    matplotlib==3.10.6 \
    seaborn==0.13.2 \
    scikit-learn==1.7.1 \
    tqdm==4.67.1 \
    requests==2.32.5 \
    litellm==1.78.5

Download sample search engine


In [19]:
!test -f minsearch.py || wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

Import packages


In [ ]:
import json

import litellm
import minsearch
import requests
from dotenv import load_dotenv

load_dotenv()

/home/gabrieldmenezes/Documents/repos/datadev/notebooks/python/generative-ai/rag-retrieval-search/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


True

Download documents


In [ ]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course["course"]
    for doc in course["documents"]:
        doc["course"] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Search Engine


Instance search engine


In [ ]:
index = minsearch.Index(
    text_fields=["section", "question", "text"], keyword_fields=["course"]
)

Defining question


In [ ]:
q = "The course has already started, can I still enroll?"

Vectorizing documents


In [ ]:
index.fit(documents)

Search


In [ ]:
boost = {"question": 3.0, "section": 0.5}

results = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={"course": "data-engineering-zoomcamp"},
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## Generation


In [ ]:
messages = [
    {
        "role": "user",
        "content": q,
    },
]


completion = litellm.completion(
    model="gemini/gemini-2.5-flash",
    messages=messages,
)

response = completion.choices[0].message.content
print(response)

It really **depends on several factors**, but often, **yes, you can still enroll** even if a course has already started. Many institutions have a "late enrollment" or "add/drop" period.

Here's what determines if it's possible and what you should do:

### Factors That Influence Late Enrollment:

1.  **How Long Ago Did It Start?**
    *   **A few days to a week:** Most likely to be possible. You'll have less material to catch up on.
    *   **Two to three weeks:** Still potentially possible, but you'll have more work to do to catch up. The institution might have specific "add/drop" deadlines that fall around this time.
    *   **A month or more:** Less likely, unless it's a very flexible or self-paced course. You might have missed too much foundational material or too many assignments.

2.  **Course Type and Structure:**
    *   **Self-paced online courses:** Often the most flexible, allowing enrollment at almost any time, as long as you complete by the deadline.
    *   **Asynchronous 

Prompt template


In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the question based on the CONTEXT.
Use only the facts from the CONTEXT when answering the QUESTION.
If the context doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
"""

Context


In [ ]:
context = ""

for doc in results:
    context = (
        context
        + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    )

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
messages = [
    {
        "role": "user",
        "content": prompt,
    },
]


completion = litellm.completion(
    model="gemini/gemini-2.5-flash",
    messages=messages,
)

response = completion.choices[0].message.content
print(response)

Yes, even if you don't register, you're still eligible to submit the homeworks. However, be aware that there will be deadlines for turning in the final projects.


Define search function


In [ ]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results

Search sample


In [ ]:
results = search("How do I run Kafka?")
results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the question based on the CONTEXT.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """
    context = ""

    for doc in results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
def llm(prompt):
    messages = [
        {
            "role": "user",
            "content": prompt,
        },
    ]

    completion = litellm.completion(
        model="gemini/gemini-2.5-flash",
        messages=messages,
    )

    return completion.choices[0].message.content

In [ ]:
query = "How do I run kafka?"


def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer


print(rag(query))

To run Kafka-related applications:

*   **For Java Kafka applications** (like producers, consumers, kstreams), navigate to the project directory and execute a command similar to this:
    `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`

*   **For Python Kafka applications**, first ensure that Docker images (presumably containing Kafka) are up and running. Then, it's recommended to use a virtual environment.
    To set up and activate a virtual environment (run once to set up, then activate each time):
    1.  Create the virtual environment: `python -m venv env`
    2.  Activate it: `source env/bin/activate` (or `env/Scripts/activate` for Windows)
    3.  Install requirements: `pip install -r ../requirements.txt`


## Elasticsearch

```sh
docker run -it \
  --name elasticsearch-debug \
  -p 9200:9200 \
  -p 9300:9300 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "ES_JAVA_OPTS=-Xms512m -Xmx512m" \
  docker.elastic.co/elasticsearch/elasticsearch:8.5.1

```
